<a href="https://colab.research.google.com/github/ElisaSuarez/czechitas-section3/blob/main/ConvolutionalNeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
archive = zipfile.ZipFile('archive(1).zip')

for file in archive.namelist():
        archive.extract(file, '.')

In [ ]:
import numpy as np
from scipy.io import loadmat

import matplotlib.pyplot as plt

mnist = loadmat('homNIST_test.mat')
mnist['img_data'].astype(np.float32)
list(mnist.keys())

In [ ]:
mnist['img_data'].shape

In [ ]:
mnist['labels'].shape

In [ ]:
mnist['hom_matrices'].shape

In [ ]:
plt.figure()
plt.imshow(mnist['img_data'][0,:,:], cmap='gray')
plt.show()

In [ ]:
first_digits = [np.argmax(mnist['labels']==i) for i in range(10)]
first_digits

In [ ]:
fig, axs = plt.subplots(2,5)

for i,v in enumerate(first_digits):
    if i<5:
        axs[0][i].imshow(mnist['img_data'][v,:,:], cmap='gray')
    else:
        axs[1][i-5].imshow(mnist['img_data'][v,:,:], cmap='gray')

plt.show()

In [ ]:
examples_0 = np.argpartition(mnist['labels']==0, 10, axis=0)[-10:].squeeze()

fig, axs = plt.subplots(2,5)

for i,v in enumerate(examples_0):
    if i<5:
        axs[0][i].imshow(mnist['img_data'][v,:,:], cmap='gray')
    else:
        axs[1][i-5].imshow(mnist['img_data'][v,:,:], cmap='gray')

plt.show()

In [ ]:
examples_4 = np.argpartition(mnist['labels']==4, 10, axis=0)[-10:].squeeze()

fig, axs = plt.subplots(2,5)

for i,v in enumerate(examples_0):
    if i<5:
        axs[0][i].imshow(mnist['img_data'][v,:,:], cmap='gray')
    else:
        axs[1][i-5].imshow(mnist['img_data'][v,:,:], cmap='gray')

plt.show()

In [ ]:
examples,_ = np.nonzero(mnist['labels']==4)
examples = np.random.choice(examples, 10)

fig, axs = plt.subplots(2,5)

for i,v in enumerate(examples):
    if i<5:
        axs[0][i].imshow(mnist['img_data'][v,:,:], cmap='gray')
    else:
        axs[1][i-5].imshow(mnist['img_data'][v,:,:], cmap='gray')

plt.show()

In [ ]:
import cv2 as cv

kernel = np.array([[-1,-2,-1],
                   [0,0,0],
                   [1,2,1]],np.float32)
out1 = [cv.filter2D(mnist['img_data'][v,:,:],-1,kernel) for v in examples]
for i,img in enumerate(out1):
    temp = img
    temp[temp<0] = 0
    temp[temp>1] = 1
    out1[i] = temp

kernel2 = np.array([[-1,0,1],
                   [-2,0,2],
                   [-1,0,1]],np.float32)
out2 = [cv.filter2D(mnist['img_data'][v,:,:],-1,kernel2) for v in examples]
for i,img in enumerate(out2):
    temp = img
    temp[temp<0] = 0
    temp[temp>1] = 1
    out2[i] = temp

    
kernel3 = np.array([[-2,-1,0],
                   [-1,1,1],
                   [-0,1,2]],np.float32)
out3 = [cv.filter2D(mnist['img_data'][v,:,:],-1,kernel3) for v in examples]
for i,img in enumerate(out3):
    temp = img
    temp[temp<0] = 0
    temp[temp>1] = 1
    out3[i] = temp


out4 = [np.stack([o1,o2,o3], axis=2) for o1,o2,o3 in zip(out1, out2, out3)]
    
fig, axs = plt.subplots(4,10, figsize=(20,8), sharex=True, sharey=True)

for i,v in enumerate(out1+out2+out3 + out4):
    if i<10:
        axs[0][i].imshow(v, cmap='gray')
    elif i<20:
        axs[1][i-10].imshow(v, cmap='gray')
    elif i<30:
        axs[2][i-20].imshow(v, cmap='gray')
    else:
        axs[3][i-30].imshow(v)

plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(
    [
        keras.Input(shape=(40,40,1)),
        layers.Conv2D(
            filters=5,
            kernel_size=3,
            activation="relu"),
        layers.Conv2D(
            filters=10,
            kernel_size=3,
            activation="relu"),
        layers.MaxPooling2D(3),
        layers.Conv2D(
            filters=20,
            kernel_size=3,
            activation="relu"),
        layers.GlobalMaxPooling2D(),
        layers.Dense(10,activation="softmax"),
        
    ]
)

print('------------------------------MODEL TO TRAIN----------------------------------')
print(model.summary())


In [ ]:
feature_extractor = keras.Model(
    inputs=model.inputs,
    outputs=[layer.output for layer in model.layers],
)

print('--------------------------MODEL TO SEE FEATURES-------------------------------')
print(feature_extractor.summary())

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(mnist['img_data'],mnist['labels'])
print('Train Set Size: ', x_train.shape[0])
print('Test Set Size: ', x_test.shape[0])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)



history = model.fit(
    x_train,
    tf.squeeze(tf.one_hot(y_train,10)),
    batch_size=100,
    epochs=3,
    validation_data=(
        x_test,
        tf.squeeze(tf.one_hot(y_test,10))
    )
)

In [ ]:
idx = 5
features = feature_extractor(x_test[idx:idx+1,:,:])

plt.figure()
plt.imshow(tf.squeeze(x_test[idx:idx+1,:,:]), vmin=0, vmax=1)
plt.show()

In [ ]:
layer = 0
n_maps = features[layer].shape[3]
fig, axs = plt.subplots(1,n_maps, figsize=(15,5), sharex=True, sharey=True)

for i in range(n_maps):
    axs[i].imshow(features[layer][0,:,:,i], vmin=np.min(features[layer]),vmax=np.max(features[layer]))
plt.show()

In [ ]:
layer = 1
n_maps = features[layer].shape[3]
fig, axs = plt.subplots(1,n_maps, figsize=(15,5), sharex=True, sharey=True)

for i in range(n_maps):
    axs[i].imshow(features[layer][0,:,:,i], vmin=np.min(features[layer]),vmax=np.max(features[layer]))
plt.show()

In [ ]:
layer = 2
n_maps = features[layer].shape[3]
fig, axs = plt.subplots(1,n_maps, figsize=(15,5), sharex=True, sharey=True)

for i in range(n_maps):
    axs[i].imshow(features[layer][0,:,:,i], vmin=np.min(features[layer]),vmax=np.max(features[layer]))
plt.show()

In [ ]:
layer = 3
n_maps = features[layer].shape[3]
fig, axs = plt.subplots(1,n_maps, figsize=(15,5), sharex=True, sharey=True)

for i in range(n_maps):
    axs[i].imshow(features[layer][0,:,:,i], vmin=np.min(features[layer]),vmax=np.max(features[layer]))
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.stem(tf.squeeze(features[4]))
plt.show()

In [ ]:
y_test[idx]

In [ ]:
plt.figure(figsize=(15,5))
plt.stem(tf.squeeze(features[5]))
plt.stem(y_test[idx], [-1], markerfmt='x')
plt.show()